# Master Thesis Analysis Report

Analysis of the participants using two different UI and comparing their pupil actitivty...

In [1]:
import pandas as pd
# from data_manager.data_manager import process_dataframe, split_data
# from analysis.analysis import score_sueq, score_sus, score_tlx, visualize_questionnaire_boxplots
from data_manager.manager import DataManager


## Getting data ready

Reading data from a CSV file and changing values to numeric; column 1 is user ID, column 2 is category (1 - Garmin, 2 - Strava)

In [2]:
data_manager = DataManager(
    responses_data_path="anketa_working.csv", 
    pupil_data_path=None
    )
cleaned_data = data_manager.get_clean_data()
cleaned_data.head(3)

,user_id,UI,Q4a,Q4b,Q4c,Q4d,Q4e,Q4f,Q4g,Q4h,...,Q6g,Q6h,Q6i,Q6j,Q8,Q9,Q10,Q11,Q12,Q13
0,83213,1,5,5,6,6,6,6,4,3,...,4,2,2,3,5,0,3,2,8,7
1,123456,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,29622,2,5,6,6,2,5,6,5,5,...,4,4,1,4,12,0,5,12,15,7


In [3]:
sueq, sus, tlx = data_manager.split_data()
sueq
sus
tlx.head(2)

,user_id,UI,Q8,Q9,Q10,Q11,Q12,Q13
0,83213,1,5,0,3,2,8,7
1,123456,1,-1,-1,-1,-1,-1,-1


In [ ]:
from analysis.analyzer import AnalysisManager

analysis_manager = AnalysisManager(
    sueq=sueq,
    sus=sus,
    tlx=tlx
)

analysis_manager.score_tlx()

,User ID,Interface Category,Mean TLX Score
0,83213,1,4.166667
1,123456,1,-1.000000
2,29622,2,8.500000
3,29622,1,4.666667
4,94249,2,0.666667
5,94249,1,1.333333
6,63367,2,0.000000
7,63367,1,8.500000
8,82586,2,1.166667
9,82586,1,2.833333


In [7]:
processed_data = process_dataframe(data)

In [4]:
mapping_dict = {
    "Q1": "Vpišite svojo raziskovalno šifro",
    "Q2": "Vmesnik, na katerem ste nazadnje reševali nalogo:",
    "Q3": "Ali ste že kdaj reševali podobne naloge?",
    "Q4": "Ali ste že kdaj reševali podobne naloge?",
    "Q5": "Ali ste že kdaj reševali podobne naloge?",
    "Q6": "Ali ste že kdaj reševali podobne naloge?",
    "Q7": "Ali ste že kdaj reševali podobne naloge?",
    "Q8": "Ali ste že kdaj reševali podobne naloge?",
    "Q9": "Ali ste že kdaj reševali podobne naloge?",
    "Q10": "Ali ste že kdaj reševali podobne naloge?",
    "Q11": "Ali ste že kdaj reševali podobne naloge?",
        
        }

In [8]:
sueq, sus, tlx =split_data(processed_data)
visualized_data_tlx

NameError: name 'visualized_data_tlx' is not defined

In [8]:
tlx_scores=score_tlx(tlx).round(2)
tlx_scores

,User ID,Interface Category,Mean TLX Score
0,83213,1,4.17
1,123456,1,-1.00
2,29622,2,8.50
3,29622,1,4.67
4,94249,2,0.67
5,94249,1,1.33
6,63367,2,0.00
7,63367,1,8.50
8,82586,2,1.17
9,82586,1,2.83


In [7]:
sus_scores=score_sus(sus)

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [ ]:
nov_sus=sus.drop()

In [23]:
filtered_sueq=sueq[1:]
filtered_sueq


,Q1,Q2,Q4a,Q4b,Q4c,Q4d,Q4e,Q4f,Q4g,Q4h
1,83213,1,5,5,6,6,6,6,4,3
2,123456,1,-1,-1,-1,-1,-1,-1,-1,-1
3,29622,2,5,6,6,2,5,6,5,5
4,29622,1,6,6,6,7,3,5,6,6
5,94249,2,6,5,5,6,4,4,5,5
6,94249,1,6,6,6,6,4,4,5,5
7,63367,2,7,6,6,5,4,4,6,6
8,63367,1,3,3,4,3,5,5,6,6
9,82586,2,3,3,5,2,2,2,4,3
10,82586,1,5,3,5,5,4,6,6,4


NameError: name 'pd' is not defined

In [21]:
nasa_tlx

,Q2,Q8,Q9,Q10,Q11,Q12,Q13
0,"Vmesnik, na katerem ste nazadnje reševali nalogo:",Mentalna zahtevnost Kako mentalno zahtevna je...,Fizična zahtevnostKako fizično zahtevna je bil...,Časovna zahtevnost Kako zahteven (hiter oz. n...,Izvedba Kako uspešni ste bili pri doseganju t...,"Trud Kako zelo ste se mogli potruditi, da ste...","Frustracija Kako negotovi, nemotivirani, razd..."
1,1,5,0,3,2,8,7
2,1,-1,-1,-1,-1,-1,-1
3,2,12,0,5,12,15,7
4,1,8,0,5,5,8,2
5,2,1,0,1,0,1,1
6,1,3,0,3,0,1,1
7,2,0,0,0,0,0,0
8,1,11,0,7,6,12,15
9,2,0,0,2,0,0,5


TypeError: can only concatenate str (not "int") to str

: 

: 

: 

: 

: 

: 

1. Uploading Excel Data :

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

Participants had to fill out S-UEQ, SUS and RAW-TLX after a task on Garmin and a task on Strava. Each participant represents 2 rows in a dataframe - one row represents self-reports for Garmin (1) and one for Strava (2). To do an analysis, data has to be split into 3 distinct questionnaires.

2. split data into questionnaires 

In [3]:
split_dataframes = manager.split_data(data)

sueq_df = split_dataframes['S_UEQ']
sus_df = split_dataframes['SUS']
tlx_df = split_dataframes['NASA-TLX']
sueq_df.head()

NameError: name 'data' is not defined

: 

3. calculate scores for each questionnaire

In [ ]:
print(sueq_df.dtypes)  # Replace with the relevant DataFrame


: 

: 

: 

: 

: 

In [ ]:
import os
print(os.getcwd())


: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 